In [1]:
!pip install autokeras

In [2]:
import pandas as pd

filepath = "/home/jovyan/shared-public/ml_swe_monitoring_prod/snotel_ghcnd_stations_4yrs_all_cols_log10.csv"
chunksize = 1e5

columns = pd.read_csv(filepath, nrows=100).columns
chunked_data = pd.read_csv(filepath, chunksize=chunksize)
sample_data = pd.DataFrame(columns=columns)

for chunk in chunked_data:
    chunk.columns = columns
    chunk = chunk.sample(frac=0.05)
    sample_data = pd.concat([sample_data, chunk])

/tmp/ipykernel_4798/1122208404.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sample_data = pd.concat([sample_data, chunk])


In [3]:
import numpy as np

chosen_columns = [
  "swe_value",
  'fsca',
  'air_temperature_tmmx', 
  'air_temperature_tmmn', 
  'potential_evapotranspiration', 
  'relative_humidity_rmax', 
  'Elevation',	
  'Slope',	
  'Curvature',	
  'Aspect',	
  'Eastness',	
  'Northness',
]


sample_data = sample_data[sample_data.swe_value != -999]
sample_data = sample_data[chosen_columns].dropna()

y = np.expand_dims(sample_data["swe_value"], axis=-1)
X = sample_data.drop(columns=["swe_value"])

In [4]:
print(y.shape)
print(X.shape)

(207845, 1)
(207845, 11)


In [5]:
import autokeras as ak
import tensorflow as tf

train_data = tf.data.Dataset.from_tensor_slices((X, y))

2024-08-22 22:26:15.603955: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-22 22:26:15.621239: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-22 22:26:15.626159: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-22 22:26:15.638716: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-22 22:26:16.682223: W tensorflow/comp

In [6]:
next(train_data.take(1).as_numpy_iterator())

(array([ 0.00000000e+00,  2.93900000e+02,  2.70600000e+02,  8.20000000e+00,
         5.20000000e+01,  1.64263010e+03,  8.99886700e+01, -1.38636070e+04,
         1.20569664e+02, -7.10852100e-01, -4.70492540e-01]),
 array([0.]))

In [7]:
input_node = ak.Input()
norm = ak.Normalization()(input_node)
dense = ak.DenseBlock()(norm)
output_node = ak.RegressionHead()(dense)

auto_model = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=10
)

In [8]:
auto_model.fit(train_data, batch_size=128, epochs=10)

Trial 10 Complete [00h 01m 40s]
val_loss: 37.16086196899414

Best val_loss So Far: 36.32413864135742
Total elapsed time: 00h 17m 28s
Epoch 1/10
1624/1624 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 170.5420 - mean_squared_error: 170.5420
Epoch 2/10
1624/1624 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 12.1436 - mean_squared_error: 12.1436
Epoch 3/10
1624/1624 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 12.1177 - mean_squared_error: 12.1177
Epoch 4/10
1624/1624 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 12.9758 - mean_squared_error: 12.9758
Epoch 5/10
1624/1624 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 12.6312 - mean_squared_error: 12.6312
Epoch 6/10
1624/1624 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 14.7313 - mean_squared_error: 14.7313
Epoch 7/10
1624/1624 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 12.0481 - mean_squared_error: 12.0481
Epoch 8/10
1624/1624 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 12.2792 - mean_squared_error: 12.2792
Epoch 9/10
1624/1624 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 12.